In [4]:
import xml.etree.ElementTree as ET
import nltk
from nltk.corpus import stopwords

polish_stopwords = set(stopwords.words('polish'))
folder = 'NIE'

tree = ET.parse('./polish/' + folder + '/ann_named.xml')
root = tree.getroot()

mapping = {
     'orgName': 'ORGANIZATION',
     'persName': 'PERSON',
     'geogName': 'GEOGRAPHY',
     'placeName': 'PLACE',
     'date': 'DATE',
     'time': 'TIME'
     }

named_entities = {}

for t in root.iter('{http://www.tei-c.org/ns/1.0}seg'):
    ne_type = ''
    ne_value = ''

    for z in t.iter("{http://www.tei-c.org/ns/1.0}f"):
        if z.attrib['name'] == 'type':
            ne_type = z.find('{http://www.tei-c.org/ns/1.0}symbol').attrib['value']
        if z.attrib['name'] == 'orth':
            ne_value = z.find('{http://www.tei-c.org/ns/1.0}string').text
    named_entities[ne_value] = mapping[ne_type]

r = nltk.corpus.reader.NKJPCorpusReader(root='./corpus/', 
                                        fileids=folder)
with open('./ner-polish.tsv', 'w') as f:
    for w in r.words():
        if w.lower() in polish_stopwords:
            continue
        if w in named_entities.keys():
            f.write("{}\t{}".format(w.encode("utf-8"), named_entities[w]))
        else:
            f.write("{}\t{}".format(w.encode("utf-8"), "O"))
        



*Buiding the model*

#prop.txt - file
```
trainFile = train/ner-polish.tsv
serializeTo = dummy-ner-model-polish.ser.gz
map = word=0,answer=1

useClassFeature=true
useWord=true
useNGrams=true
noMidNGrams=true
maxNGramLeng=6
usePrev=true
useNext=true
useSequences=true
usePrevSequences=true
maxLeft=1
useTypeSeqs=true
useTypeSeqs2=true
useTypeySequences=true
wordShape=chris2useLC
useDisjunctive=true
```

run java stanford trainer

```
java -cp "stanford-ner.jar:lib/*" -mx4g edu.stanford.nlp.ie.crf.CRFClassifier -prop train/prop.txt
```

Then you can test your train model in python

In [6]:
import nltk
from nltk.tag.stanford import StanfordNERTagger

# Optional
import os


sentence = u'4.01.1928 urodził się Tadeusz Dominik - jeden z najwybitniejszych malarzy polskich. Wiewiórki nam doniosły, że wygląda na lat pięćdziesiąt. No, to życzymy jeszcze pięćdziesięciu!'

jar = './ner/stanford-ner/stanford-ner.jar'
model = './ner/stanford-ner/dummy-ner-model-polish.ser.gz'

ner_tagger = StanfordNERTagger(model, jar, encoding='utf8')

words = nltk.word_tokenize(sentence)
print (words)
print(ner_tagger.tag(words))


['4.01.1928', 'urodził', 'się', 'Tadeusz', 'Dominik', '-', 'jeden', 'z', 'najwybitniejszych', 'malarzy', 'polskich', '.', 'Wiewiórki', 'nam', 'doniosły', ',', 'że', 'wygląda', 'na', 'lat', 'pięćdziesiąt', '.', 'No', ',', 'to', 'życzymy', 'jeszcze', 'pięćdziesięciu', '!']
[('4.01.1928', 'O'), ('urodził', 'O'), ('się', 'O'), ('Tadeusz', 'PERSON'), ('Dominik', 'PERSON'), ('-', 'O'), ('jeden', 'O'), ('z', 'O'), ('najwybitniejszych', 'O'), ('malarzy', 'O'), ('polskich', 'O'), ('.', 'O'), ('Wiewiórki', 'O'), ('nam', 'O'), ('doniosły', 'O'), (',', 'O'), ('że', 'O'), ('wygląda', 'O'), ('na', 'O'), ('lat', 'O'), ('pięćdziesiąt', 'O'), ('.', 'O'), ('No', 'O'), (',', 'O'), ('to', 'O'), ('życzymy', 'O'), ('jeszcze', 'O'), ('pięćdziesięciu', 'O'), ('!', 'O')]
